In [ ]:
# pip install -qU duckduckgo-search langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import PyPDF2
import faiss
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain_community.llms import Ollama
from langchain_core.tools import tool
from crewai import Agent, Task, Crew,LLM
from langchain_community.tools import DuckDuckGoSearchRun,DuckDuckGoSearchResults
from langchain_core.tools import BaseTool
from groq import Groq
from crewai.tools import tool
from litellm import completion

# using ollama

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# client=Groq(api_key="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

llm1=LLM(
    model="ollama/llama3.2:1b",
    base_url="http://localhost:11434"  
)
# llm1 = LLM(
#     model="groq/llama-3.1-70b-versatile",
#     api_key="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
#  )



# llm1 = LLM(
#     model="llama3.2:1b",           
#     base_url="http://localhost:11434",
#     litellm_provider="ollama"      
# )

# llm = Ollama(model="llama3.2:1b")

def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted
            return text
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return ""

def chunk_text(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    return text_splitter.split_text(text)

def create_vector_store(text_chunks, embedder):
    embeddings = embedder.encode(text_chunks, show_progress_bar=True)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index, embeddings, text_chunks

In [15]:
from crewai.tools import tool
@tool("retrieving coherent chunks")
def retrieve_from_pdf(query: str) -> str:
    """Retrieve relevant information from the uploaded PDF."""
    if vector_store is None:
        return "No PDF loaded."
    docs = vector_store.similarity_search(query, k=5)
    return "\n".join([doc.page_content for doc in docs])

search_tool=DuckDuckGoSearchRun()

# retrieve_from_pdf_tool = Tool(
#     name="PDFRetriever",
#     func=retrieve_from_pdf,
#     description="Retrieve relevant information from the uploaded PDF given a user query."
# )
# search_tool1 = Tool(
#     name="DuckDuckGoSearch",
#     func=search_tool.run,
#     description="Search the web using DuckDuckGo to find information related to the user query."
# )


# print(isinstance(retrieve_from_pdf_tool, BaseTool))  
# print(isinstance(search_tool_wrapped, BaseTool))     



In [16]:
# query_agent=Agent(
#     role="to understand the context of user query:{query}",
#     goal="rewrite and elaborate the user query:{query} for efficient rerieval of the context",
#     backstory="you are expert in understanding contextual meaning of the user query:{query}",
#     llm =llm1,
#     reasoning=True,
#     Verbose=True
# )
retriever_agent=Agent(
    role="Retrieve relevant information to answer the user query: {query}",
    goal="Retrieve the most relevant information from the available sources  for the user query: {query}, always try to use the pdf search tool first If you are not able to retrieve the information from the pdf search tool then try to use the web search tool", 
    backstory="You help extract useful info from user-uploaded documents.",
    llm =llm1,
    tools=[retrieve_from_pdf],
    reasoning=True,
    Verbose=True
)
answering_agent = Agent(
    role="Genrating coherent answers",
    goal="Generate clear and helpful answers using the retrieved context",
    backstory="You are an expert at synthesizing information and answering complex user queries.",
    verbose=True,
    llm=llm1
)




In [7]:
# task1=Task(
#     description="understand the user query:{query} and rewrite it with additioal contextual information",
#     expected_output="contextual rich query:{query} to be used information retrieval",
#     agent=query_agent
# )
task2=Task(
    description="to fetch information based upon the user query:{query} contextual understanding",
    expected_output="precise information related to user query:{query} ",
    agent=retriever_agent
)

task3=Task(
    description="generate precise response based upon user query:{query},following the constraints defined by the user",
    expected_output="generated precise and coherent answer and if not able to generate then respond  sorry not able to find answers ",
    agent=answering_agent
)

In [ ]:
#  os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"



In [17]:
crew=Crew(
    agents=[retriever_agent,answering_agent],
    tasks=[task2,task3],
    verbose=True,
    memory=False
    
)

In [ ]:
!pip install pyPDF2

Note: you may need to restart the kernel to use updated packages.


In [18]:
pdf_files = input("Enter the paths to your PDF files: ").split(",")
pdf_files = [f.strip() for f in pdf_files]

if pdf_files:
    print("Processing PDFs...")
    all_chunks = []
    for file_path in pdf_files:
        if os.path.exists(file_path):
            text = extract_text_from_pdf(file_path)
            if text:
                chunks = chunk_text(text)
                all_chunks.extend(chunks)
            else:
                print(f"No text extracted from {file_path}")
        else:
            print(f"File not found: {file_path}")

    
    if all_chunks:
        vector_store, embeddings, documents = create_vector_store(all_chunks, embedder)
        print("Documents processed and vector store created!")
    else:
        print("No valid documents processed.")
        exit()
else:
    print("No PDF files provided.")
    exit()

Enter the paths to your PDF files:  agentic rag.pdf


Processing PDFs...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Documents processed and vector store created!


In [19]:
 while True:
        query = input("\n Ask a question about the PDF (or type 'exit'): ")
        if query.lower() in ["exit", "quit"]:
            break
        result = crew.kickoff(inputs={"query": query})
        print("\n Answer:\n", result)



 Ask a question about the PDF (or type 'exit'):  explain the document


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 587da218-b079-43e7-a9b6-21a916c9b19a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new

LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new

LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.APIConnectionError: OllamaException - litellm.Timeout: Connection timed out after 600.0         │
│  seconds.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

# using groq api

In [5]:


# from crewai import tool
from duckduckgo_search import DDGS

@tool("duckduckgo_web_search")
def duckduckgo_search(query: str) -> str:
    """Searches the web using DuckDuckGo and returns top 5 results."""
    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=5)
            output = ""
            for i, r in enumerate(results, 1):
                output += f"{i}. {r['title']} ({r['href']})\n{r['body']}\n\n"
            return output or "No results found."
    except Exception as e:
        return f"Error during search: {e}"


In [ ]:
!pip install PyMuPDF


   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
    ----------------

In [22]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import io
import numpy as np
import fitz 

device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize CLIP model and processor for image embeddings
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# Text embedding model (sentence-transformers)
# text_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
text_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/clip-ViT-B-32")
 # PyMuPDF

def extract_images_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    image_bytes_list = []

    for page in doc:
        for img in page.get_images(full=True):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_bytes_list.append(image_bytes)

    return image_bytes_list



def embed_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    # Normalize the embeddings (optional but recommended)
    image_embedding = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
    return image_embedding.cpu().numpy()[0]

def build_faiss_from_text_and_images(text, list_of_image_bytes):
    # 1. Text processing
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_text(text)
    text_documents = [Document(page_content=chunk, metadata={"type": "text"}) for chunk in chunks]

    # Embed text docs (using langchain's text_embedder)
    vector_store = FAISS.from_documents(text_documents, text_embedder)

    # 2. Process images: create Document objects with CLIP embeddings
    image_docs = []
    image_embeddings = []
    for i, img_bytes in enumerate(list_of_image_bytes):
        emb = embed_image(img_bytes)
        image_embeddings.append(emb)
        image_docs.append(Document(page_content=f"<Image {i}>", metadata={"type": "image", "index": i}))

    if image_embeddings:
        # Convert embeddings to numpy array
        image_embs_np = np.array(image_embeddings).astype('float32')

        # Add image embeddings to the FAISS index
        vector_store.index.add(image_embs_np)
        vector_store.add_documents(image_docs, embeddings=image_embs_np)# Add image docs metadata
  
 

    return vector_store




In [14]:

from langchain.embeddings import HuggingFaceEmbeddings




def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        content = page.extract_text()
        if content:
            text += content + "\n"
    return text


# def build_faiss_from_text(text):
#     splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
#     chunks = splitter.split_text(text)
#     documents = [Document(page_content=chunk) for chunk in chunks]
#     embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     vector_store = FAISS.from_documents(documents, embedding_model)
#     return vector_store


# llm1=LLM(
#     model="ollama/llama3.2:1b",
#     base_url="http://localhost:11434"  
# )
llm1 = LLM(
    model="groq/meta-llama/llama-4-scout-17b-16e-instruct",
    api_key="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"
 )


session_vector_store = None


@tool
def retrieve_from_pdf(query: str) -> str:
    """Retrieve relevant information from the uploaded PDF."""
    if session_vector_store is None:
        return "No PDF loaded."
    docs = session_vector_store.similarity_search(query, k=5)
    return "\n".join([doc.page_content for doc in docs])



retriever_agent = Agent(
    role="Retriever",
    goal="Search the uploaded PDF and extract relevant context",
    backstory="You help extract useful info from user-uploaded documents.",
    tools=[retrieve_from_pdf,duckduckgo_search],
    llm=llm1,
    verbose=True
)

responder_agent = Agent(
    role="Responder",
    goal="Answer the user question based on PDF content",
    backstory="You use the retrieved content to answer the user's query.",
    llm=llm1,
    verbose=True
)




In [23]:
def run_pdf_agent_rag(query, vector_store):
    global session_vector_store
    session_vector_store = vector_store

    retrieval_task = Task(
        description=f"Retrieve context from the uploaded PDF for: '{query}'",
        agent=retriever_agent,
        expected_output="A block of relevant information"
    )

    response_task = Task(
        description=f"Answer the user question: '{query}' using the retrieved context.",
        agent=responder_agent,
        expected_output="A well-structured answer to the user's query"
    )

    crew = Crew(
        agents=[retriever_agent, responder_agent],
        tasks=[retrieval_task, response_task],
        verbose=True
    )

    result = crew.kickoff(inputs={"query": query})
    return result


def main():
    
    pdf_path = input("Enter path to your PDF file: ").strip()

    if not os.path.exists(pdf_path):
        print("PDF file not found.")
        return

    print("Extracting text from PDF...")
    text = extract_text_from_pdf(pdf_path)

    if not text.strip():
        print(" No text extracted.")
        return

    print("Extracting image from PDF...")
    images= extract_images_from_pdf(pdf_path)

    if not text.strip():
        print(" No text extracted.")
        return

    print(" Embedding PDF into FAISS...")
    vector_store =build_faiss_from_text_and_images(text,images)
    print(" PDF ready for question answering.")

    while True:
        query = input("\n Ask a question about the PDF (or type 'exit'): ")
        if query.lower() in ["exit", "quit"]:
            break
        result = run_pdf_agent_rag(query, vector_store)
        print("\n Answer:\n", result)

if __name__ == "__main__":
    main()

Enter path to your PDF file:  attention is all you need.pdf


Extracting text from PDF...
Extracting image from PDF...
 Embedding PDF into FAISS...
 PDF ready for question answering.



 Ask a question about the PDF (or type 'exit'):  explain me about transformer architecture with image


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: aca4389c-8fa7-4c9e-82ca-051081a3bb6d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Retriever                                                                                               │
│                                                                                                                 │
│  Task: Retrieve context from the uploaded PDF for: 'explain me about transformer architecture with image'       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Responder                                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The Transformer model architecture, as depicted in Figure1, consists of two main components: the Encoder and   │
│  the Decoder. These components are stacked with multiple layers, leveraging self-attention mechanisms and       │
│  fully connected layers.                                                                                        │
│                                                                                                                 │
│  ### Encoder                                                                                                    │
│                                                                                                                 │
│  The Encoder is composed of a stack of **N=6 identical layers**. Each layer in the encoder has two sub-layers:  │
│  1. **Multi-head self-attention mechanism**: This allows the model to attend to different parts of the input    │
│  sequence simultaneously and weigh their importance.                                                            │
│  2. **Position-wise fully connected feed-forward network**: This applies a fully connected feed-forward         │
│  network to each position separately and identically.                                                           │
│                                                                                                                 │
│  The Transformer employs **residual connections** around each of the two sub-layers, followed by **layer        │
│  normalization**. The output of each sub-layer is computed as `LayerNorm(x + Sublayer(x))`, where               │
│  `Sublayer(x)` is the function implemented by the sub-layer itself. This design facilitates the training of     │
│  deeper networks.                                                                                               │
│                                                                                                                 │
│  All sub-layers in the model, as well as the embedding layer, output a vector of dimensionality **d_model =     │
│  512**. This uniformity in output dimensions simplifies the design and implementation of the model.             │
│                                                                                                                 │
│  ### Decoder                                                                                                    │
│                                                                                                                 │
│  Although not detailed in the provided text, the Decoder, shown in the right half of Figure1, similarly         │
│  utilizes stacked layers with self-attention mechanisms and point-wise, fully connected layers. The Decoder's   │
│  architecture is designed to generate output sequences, taking into account the output from the Encoder.        │
│                                                                                                                 │
│  ### Transformer Architecture Overview                                                                          │
│                                                                                                                 │
│  In summary, the Transformer architecture relies on stacked self-attention and point-wise, fully connected      │
│  layers for both the Encoder and Decoder. This design a

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8ebfc658-ea3b-4d28-b540-6ec812a3faf7                                                                     │
│  Agent: Responder                                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: aca4389c-8fa7-4c9e-82ca-051081a3bb6d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The Transformer model architecture, as depicted in Figure1, consists of two main components:     │
│  the Encoder and the Decoder. These components are stacked with multiple layers, leveraging self-attention      │
│  mechanisms and fully connected layers.                                                                         │
│                                                                                                                 │
│  ### Encoder                                                                                                    │
│                                                                                                                 │
│  The Encoder is composed of a stack of **N=6 identical layers**. Each layer in the encoder has two sub-layers:  │
│  1. **Multi-head self-attention mechanism**: This allows the model to attend to different parts of the input    │
│  sequence simultaneously and weigh their importance.                                                            │
│  2. **Position-wise fully connected feed-forward network**: This applies a fully connected feed-forward         │
│  network to each position separately and identically.                                                           │
│                                                                                                                 │
│  The Transformer employs **residual connections** around each of the two sub-layers, followed by **layer        │
│  normalization**. The output of each sub-layer is computed as `LayerNorm(x + Sublayer(x))`, where               │
│  `Sublayer(x)` is the function implemented by the sub-layer itself. This design facilitates the training of     │
│  deeper networks.                                                                                               │
│                                                                                                                 │
│  All sub-layers in the model, as well as the embedding layer, output a vector of dimensionality **d_model =     │
│  512**. This uniformity in output dimensions simplifies the design and implementation of the model.             │
│                                                                                                                 │
│  ### Decoder                                                                                                    │
│                                                                                                                 │
│  Although not detailed in the provided text, the Decoder, shown in the right half of Figure1, similarly         │
│  utilizes stacked layers with self-attention mechanisms and point-wise, fully connected layers. The Decoder's   │
│  architecture is designed to generate output sequences, taking into account the output from the Encoder.        │
│                                                                                                                 │
│  ### Transformer Architecture Overview                                                                          │
│                                                                                                                 │
│  In summary, the Transformer architecture relies on st


 Answer:
 The Transformer model architecture, as depicted in Figure1, consists of two main components: the Encoder and the Decoder. These components are stacked with multiple layers, leveraging self-attention mechanisms and fully connected layers.

### Encoder

The Encoder is composed of a stack of **N=6 identical layers**. Each layer in the encoder has two sub-layers:
1. **Multi-head self-attention mechanism**: This allows the model to attend to different parts of the input sequence simultaneously and weigh their importance.
2. **Position-wise fully connected feed-forward network**: This applies a fully connected feed-forward network to each position separately and identically.

The Transformer employs **residual connections** around each of the two sub-layers, followed by **layer normalization**. The output of each sub-layer is computed as `LayerNorm(x + Sublayer(x))`, where `Sublayer(x)` is the function implemented by the sub-layer itself. This design facilitates the training of de


 Ask a question about the PDF (or type 'exit'):  exit
